# Problema de dimensionamento de lotes

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB

## Descrição do problema

Uma fábrica precisa encontrar um planejamento de produção de um determinado item para atende demandas em um horizonte de n períodos.

Em cada período $t$ existe uma demanda $d_t$ que pode ser atendida por produção no período $t$ ou por itens em estoque.

Suponha que no início do processo de produção o estoque inicial seja igual a zero, assim como no útimo período.

Considere os seguintes dados para o problema.

- $f_t$, custo fixo de produção no período $t$
- $p_t$, custo de produção unitário no perído $t$
- $h_t$, custo de estoque unitário no período $t$
- $d_t$, quantidade da demanta no período $t$

## Formulação do problema

1. Variáveis

    - $x_i$, quantidade produzida no período $t$
    - $s_t$, quantidade em estoque no final do período $t$
    - $ y_t = 
    \begin{cases}
    1, & \text{se ocorre produção no período {\it t}} \\
    0, & \text{caso contrário}
    \end{cases}
    $

2. Restrições
    - fluxo de produção
    $
    \begin{align}\notag
    s_{t-1} + x_t & = d_t + s_t \text{ para } t=1, 2, \ldots, n
    \end{align}
    $

    - limitação de produção em $t$
    $ 
    \begin{align}\notag
    x_t & \leq (\sum_{i=t}^{n} d_i) y_t, \text{ para } t=1,2, \ldots, n
    \end{align}
    $

    - estoque inicial e final igual a zero
    $$
    s_0 = s_n = 0
    $$

    - não negatividade e integralidade
    $
    \begin{align}
    x_t & \geq 0, \text{ para } t=1,2, \ldots, n \\
    s_t & \geq 0, \text{ para } t=1,2, \ldots, n \\
    y_t & \in \{0,1\}, \text{ para } t=1,2, \ldots, n \\
    \end{align}
    $

    - função objetivo
    $$
    \min \sum_{t=1}^{n} p_t x_t + \sum_{t=1}^{n} h_t s_t + \sum_{t=1}^{n} f_t y_t
    $$

In [2]:
def readdata(datafile):
  
    with open(datafile, 'r') as file: linhas = file.readlines()

    # remove linha vazia inicial e elimina os "\n" de cada linha
    linhas = [a.strip() for a in linhas] 

    # lendo o tamanho da instancia
    aux = 0
    N = int(linhas[aux])
  
    # definindo vetores
    H = np.zeros(N)
    P = np.zeros(N)
    F = np.zeros(N)
    D = np.zeros(N)
  
    # lendo e armazenando dados
    aux = aux+1
    F[0] = float(linhas[aux])
    for i in range(1,N):
        F[i] = F[0]

    aux = aux+1
    H[0] = float(linhas[aux])
    for i in range(1,N):
        H[i] = H[0]

    aux = aux+1
    P[0] = float(linhas[aux])
    for i in range(1,N):
        P[i] = P[0]

    aux = aux+1
    D = np.fromstring(linhas[aux], dtype=float, sep = ' ')
 
    return N, H, P, F, D

In [3]:
def modelo(N, H, P, F, D, model):

    # adicionando variaveis
    x = model.addVars(N, name='x') 
    s = model.addVars(N, name='s')  
    y = model.addVars(N, vtype=GRB.BINARY, name='y') 

    # definindo funcao objetivo
    obj = 0
    for i in range(0, N):
        obj += P[i] * x[i]
        obj += H[i] * s[i]
        obj += F[i] * y[i]

    model.setObjective(obj, GRB.MINIMIZE)
  
    # definindo restrições
    model.addConstr(x[0] - s[0] == D[0])
    for i in range(1, N):
        model.addConstr(s[i-1] + x[i] - s[i] == D[i])
  
    for i in range(0, N):
        model.addConstr(x[i] - (D[i:N].sum())*y[i] <= 0)

    model.addConstr(s[N-1] == 0)

    # exportando modelo
    model.write("uls.lp")

    return model

In [4]:
def solve(model):
    # resolvendo o problema
    model.optimize()

In [5]:
if __name__ == "__main__":
    
    datafile = "../data/uls/52_2.txt"

    # lendo os dados
    N, H, P, F, D = readdata(datafile)

    # criando o modelo
    model = gp.Model("uls")

    # configurando parametros
    #model.Params.IterationLimit = 1000 # define o número de iterações do simplex
    model.Params.TimeLimit = 120 # define tempo limite
    model.Params.method = 0 #-1=automatic, 0=primal, 1=dual , 2=barrier
    #model.Params.NodeMethod = -1 #-1=automatic, 0=primal, 1=dual , 2=barrier
    model.Params.Threads = 1

    # modelando o problema
    modelo(N, H, P, F, D, model)

    # resolvendo o problema
    solve(model)
    

Set parameter Username
Academic license - for non-commercial use only - expires 2026-02-19
Set parameter TimeLimit to value 120
Set parameter Method to value 0
Set parameter Threads to value 1
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (linux64)

CPU model: Intel(R) Core(TM) i5-1035G1 CPU @ 1.00GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 1 threads

Optimize a model with 105 rows, 156 columns and 260 nonzeros
Model fingerprint: 0x5482fc9f
Variable types: 104 continuous, 52 integer (52 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [2e-01, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+01, 1e+02]
Found heuristic solution: objective 15552.000000
Presolve removed 4 rows and 4 columns
Presolve time: 0.00s
Presolved: 101 rows, 152 columns, 252 nonzeros
Variable types: 101 continuous, 51 integer (51 binary)

Root relaxation: objective 6.159820e+03, 124 iterations, 0.0

     0     0 7010.33733    0   46 14404.2000 7010.33733  51.3%     -    0s
     0     0 8665.24865    0   38 14404.2000 8665.24865  39.8%     -    0s
H    0     0                    14322.032682 8665.24865  39.5%     -    0s
H    0     0                    10228.400000 8665.24865  15.3%     -    0s
     0     0 8727.31587    0   41 10228.4000 8727.31587  14.7%     -    0s
     0     0 8731.40430    0   42 10228.4000 8731.40430  14.6%     -    0s
     0     0 8731.74671    0   42 10228.4000 8731.74671  14.6%     -    0s
     0     0 8876.13596    0   40 10228.4000 8876.13596  13.2%     -    0s
     0     0 8896.36215    0   39 10228.4000 8896.36215  13.0%     -    0s
     0     0 8896.63759    0   40 10228.4000 8896.63759  13.0%     -    0s
     0     0 9026.32004    0   31 10228.4000 9026.32004  11.8%     -    0s
     0     0 9029.25710    0   31 10228.4000 9029.25710  11.7%     -    0s
     0     0 9033.56001    0   32 10228.4000 9033.56001  11.7%     -    0s
     0     0 9034.10597  

In [7]:
# imprimindo solucao
for var in model.getVars():
    if var.x > 1e-6:
        print(f"{var.varName} = {var.x}")

x[0] = 408.0
x[4] = 421.9999999999999
x[8] = 402.0000000000043
x[12] = 370.99999999998937
x[16] = 462.99999999999966
x[21] = 495.0
x[26] = 419.9999999999973
x[30] = 403.9999999999946
x[34] = 381.0
x[38] = 414.0
x[42] = 521.9999999999993
x[47] = 450.0
s[0] = 304.0
s[1] = 196.0
s[2] = 108.0
s[4] = 316.9999999999999
s[5] = 219.99999999999991
s[6] = 112.99999999999991
s[8] = 302.0000000000043
s[9] = 195.00000000000435
s[10] = 102.00000000000435
s[12] = 263.0
s[13] = 168.0
s[14] = 84.0
s[16] = 368.99999999999966
s[17] = 282.00000000000006
s[18] = 194.00000000000006
s[19] = 103.00000000000006
s[21] = 392.0
s[22] = 298.0
s[23] = 187.99999999999994
s[24] = 90.99999999999994
s[26] = 306.9999999999973
s[27] = 209.99999999999727
s[28] = 108.99999999999909
s[30] = 288.9999999999946
s[31] = 189.00000000000028
s[32] = 98.00000000000028
s[34] = 287.0
s[35] = 194.00000000000003
s[36] = 100.00000000000003
s[38] = 306.0
s[39] = 209.00000000000003
s[40] = 105.00000000000003
s[42] = 392.9999999999993
s[43

In [8]:
# imprimindo o valor otimo
print(f"Objetivo = {model.objVal}")

Objetivo = 9263.399999999998


In [9]:
# imprime tempo de execucao
print(f'Runtime: {model.Runtime}')

Runtime: 0.7181668281555176


In [10]:
# clear model
model.dispose()